In [2]:
class Point2D:
    __slots__= ('_x', '_y')
    
    def __init__(self, x,y):
        self._x = x
        self._y = y
        
    @property
    def x(self):
        return self._x
    
    
    @property
    def y(self):
        return self._y
    
    def __eq__(self, other):
        return isinstance(other, Point) and (self.x, self.y) == (other.x, other.y)
    
    def __hash__(self):
        return hash((self.x, self.y))
    
    
    def __repr__(self):
        return f'Point2D({self.x}, {self.y})'
    
    
    def __str__(self):
        return f'({self.x}, {self.y})'
    
class Point3D:
    __slots__=('_x', '_y', '_z')
    
    
    def __init__(self, x,y,z):
        self._x = x
        self._y = y
        self._z = z
        
        
       
    @property
    def x(self):
        return self._x
    
    
    @property
    def y(self):
        return self._y
    
    
        
    @property
    def z(self):
        return self._z
    
    def __eq__(self, other):
        return isinstance(other, Point) and (self.x, self.y, self.z) == (other.x, other.y, other.z)
    
    def __hash__(self):
        return hash((self.x, self.y, self.z))
    
    
    def __repr__(self):
        return f'Point2D({self.x}, {self.y}, {self.z})'
    
    
    def __str__(self):
        return f'({self.x}, {self.y}, {self.z})'
    
    

In [4]:
class Point2D:
    _fields = ['x', 'y']
    
    def __init__(self, x,y):
        self._x = x
        self._y = y
        
class Point3D:
    _fields = ['x', 'y', 'z']
    
    def __init__(self, x,y,z):
        self._x = x
        self._y = y
        self._z = z

In [5]:
class SlottedStruct(type):
    def __new__(mcls, name, bases, class_dict):
        cls_obj = super().__new__(mcls, name, bases, class_dict)
        
        #slots
        
        setattr(cls_obj,'__slots__', [f'_{field}' for field in cls_obj._fields])
        
        #read only prop
        for field in cls_obj._fields:
            slot = f'_{field}'
            setattr(cls_obj, field, property(fget=lambda self, attrib = slot:getattr(self, attrib)))
            
            
            
            ## prop is a closure (fn) slot is coming from outer scope so lambda isa closure
            ## so every lambda is pointing to same slot through self
            ## so it is a shared ref to the same slot which means by the time we done iter through the fields
            ## lambda will have same slot val whioch is the last field value thats why we attrib
            
        return cls_obj

In [6]:
class Person(metaclass = SlottedStruct):
    _fields = ['name', 'age']
    
    def __init__(self, name, age):
        self._name = name
        self._age = age

In [7]:
vars(Person)

mappingproxy({'__module__': '__main__',
              '_fields': ['name', 'age'],
              '__init__': <function __main__.Person.__init__(self, name, age)>,
              '__dict__': <attribute '__dict__' of 'Person' objects>,
              '__weakref__': <attribute '__weakref__' of 'Person' objects>,
              '__doc__': None,
              '__slots__': ['_name', '_age'],
              'name': <property at 0x1f512e25a90>,
              'age': <property at 0x1f512e250e0>})

In [8]:
p = Person('kok', 19)

In [9]:
p.name

'kok'

In [10]:
p.age

19

In [11]:
p.name = 'lolo'

AttributeError: can't set attribute

In [14]:
class SlottedStruct(type):
    def __new__(mcls, name, bases, class_dict):
        cls_obj = super().__new__(mcls, name, bases, class_dict)
        
        #slots
        
        setattr(cls_obj,'__slots__', [f'_{field}' for field in cls_obj._fields])
        
        #read only prop
        for field in cls_obj._fields:
            slot = f'_{field}'
            setattr(cls_obj, field, property(fget=lambda self, attrib = slot:getattr(self, attrib)))
        
        #__eq__
        def eq(self, other):
            if isinstance(other, cls_obj):
                self_fields = [getattr(self, field) for field in cls_obj._fields]
                other_fields = [getattr(other, field) for field in cls_obj._fields]
                return self_fields == other_fields
            return False
        
        setattr(cls_obj, '__eq__', eq)
        
        ##hash
        
        def hash_(self):
            field_values = (getattr(self, field) for field in cls_obj._fields)
            return hash(tuple(field_values))
        setattr(cls_obj, '__hash__', hash_)
        
        
        ##__str__
        def str_(self):
            field_values = (getattr(self, field) for field in cls_obj._fields)
            field_values_join = ', '.join(map(str, field_values))
            return f'{cls_obj.__name__}({field_values_join})'
        
        
        
        setattr(cls_obj, '__str__', str_)
        
        #__repr__
        
        def repr_(self):
            field_values = (getattr(self, field) for field in self._fields)
            field_key_values = (f'{k}={v}'for k,v in zip(cls_obj._fields, field_values))
            field_key_values_str = ', '.join(field_key_values)
            return f'{self.__class__.__name__}({field_key_values_str})'
        setattr(cls_obj, '__repr__', repr_)
            
            
            
        
        
        return cls_obj

In [15]:
class Person(metaclass = SlottedStruct):
    _fields = ['name']
    
    def __init__(self, name):
        self._name = name

In [16]:
type(Person)

__main__.SlottedStruct

In [17]:
p1 = Person('alex')
p2 = Person('alex')

In [27]:
type(p1)

__main__.Person

In [28]:
repr(p1)

'Person(name=alex)'

In [30]:
class Point2D(metaclass = SlottedStruct):
    _fields = ['x', 'y']
    
    def __init__(self, x, y):
        self._x = x
        self._y =y
        
class Point3D(metaclass = SlottedStruct):
    _fields = ['x', 'y', 'z']
    
    def __init__(self, x, y, z):
        self._x = x
        self._y =y
        self._z = z

In [31]:
p1 = Point2D(1,2)
p2 =Point2D(1,2)
p3 = Point2D(0,0)

In [32]:
repr(p1), str(p1), repr(p2), str(p2), p1.x, p2.y

('Point2D(x=1, y=2)',
 'Point2D(1, 2)',
 'Point2D(x=1, y=2)',
 'Point2D(1, 2)',
 1,
 2)

In [33]:
p1 ==p2

True

In [34]:
p1==p3

False

In [35]:
Point2D.__name__, Point2D.__dict__

('Point2D',
 mappingproxy({'__module__': '__main__',
               '_fields': ['x', 'y'],
               '__init__': <function __main__.Point2D.__init__(self, x, y)>,
               '__dict__': <attribute '__dict__' of 'Point2D' objects>,
               '__weakref__': <attribute '__weakref__' of 'Point2D' objects>,
               '__doc__': None,
               '__slots__': ['_x', '_y'],
               'x': <property at 0x1f512270540>,
               'y': <property at 0x1f5122705e0>,
               '__eq__': <function __main__.SlottedStruct.__new__.<locals>.eq(self, other)>,
               '__hash__': <function __main__.SlottedStruct.__new__.<locals>.hash_(self)>,
               '__str__': <function __main__.SlottedStruct.__new__.<locals>.str_(self)>,
               '__repr__': <function __main__.SlottedStruct.__new__.<locals>.repr_(self)>}))

In [36]:
Point2D.__bases__

(object,)

In [37]:
def Struct(cls):
    return SlottedStruct(cls.__name__, cls.__bases__,dict(cls.__dict__))

In [39]:
@Struct
class Point2D():
    _fields = ['x', 'y']
    
    def __init__(self, x, y):
        self._x = x
        self._y =y
@Struct        
class Point3D():
    _fields = ['x', 'y', 'z']
    
    def __init__(self, x, y, z):
        self._x = x
        self._y =y
        self._z = z

In [40]:
type(Point2D)

__main__.SlottedStruct

In [41]:
p1 = Point2D(1,2)
p2 =Point2D(1,2)
p3 = Point2D(0,0)

In [42]:
repr(p1), str(p1), repr(p2), str(p2), p1.x, p2.y

('Point2D(x=1, y=2)',
 'Point2D(1, 2)',
 'Point2D(x=1, y=2)',
 'Point2D(1, 2)',
 1,
 2)